In [71]:
import zipfile
zip_file = "/kaggle/input/platesv2/plates.zip"

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
  zip_ref.extractall()  # extract current working directory

In [72]:
import os
import torch
from torchvision import transforms

data_dir = "/kaggle/working/plates"
train_dir = data_dir + "/train"
test_dir = data_dir + "/test"

labels = os.listdir(train_dir)[:2]
labels = ['dirty', 'cleaned']

['dirty', 'cleaned']

In [73]:
image = []
image_label = []

In [74]:
from PIL import Image
for label in labels:
    path = os.path.join(train_dir, label)
    for image_path in os.listdir(path):
        if image_path == ".DS_Store":
            continue
#         print(image_path)
        curr_image = Image.open(os.path.join(path, image_path))
#         curr_image = cv2.imread(os.path.join(path, image_path))
        image.append(curr_image)
        image_label.append(label)

In [75]:
label_map = {"dirty": 0, "cleaned": 1}
image_label = [label_map[label] for label in image_label]

In [76]:
len(image), len(image_label)

(40, 40)

In [77]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(), 
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # normalize pixel values
])

In [78]:
from torch.utils.data import Dataset
class my_dataset(Dataset):
    def __init__(self, images, labels, transform):
        self.images = images # these are features
        self.labels = labels
        self.transform = transform
    def __getitem__(self, item):
        image = self.images[item]
        image_tranformed = self.transform(image)
        label = self.labels[item]
        return image_tranformed, torch.tensor(label)
    def __len__(self):
        return len(self.labels)
        

In [79]:
data = my_dataset(image, image_label, transform)
data

In [80]:
from torch.utils.data import DataLoader
data_loader = DataLoader(data, batch_size = 20, shuffle=True)

In [81]:
from torch import nn
import torch.nn.functional as F
class kaggle_plate_model(nn.Module):
    def __init__(self):
        super(kaggle_plate_model, self).__init__()
        # Convolutional layer 1
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Convolutional layer 2
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 64 * 64, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)

        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        x = x.view(-1, 32 * 64 * 64)
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [82]:
model = kaggle_plate_model()
model

kaggle_plate_model(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=131072, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [83]:
from torch import optim
lr = 0.001
epoch = 100
optimizer = optim.Adam(model.parameters(), lr)
criterion = nn.CrossEntropyLoss()

In [84]:
from tqdm.auto import tqdm

In [85]:
for i in tqdm(range(epoch)):
    for image, label in data_loader:
        
        optimizer.zero_grad()
        
        # forward pass
        pred = model(image)
        loss = criterion(pred, label)
        
        # backward pass
        loss.backward()
        optimizer.step()
    print(f"Epoch : {i+1}, Loss : {loss}")

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch : 1, Loss : 2.5941154956817627
Epoch : 2, Loss : 11.03217601776123
Epoch : 3, Loss : 0.5118448138237
Epoch : 4, Loss : 1.5913556814193726
Epoch : 5, Loss : 1.1186578273773193
Epoch : 6, Loss : 0.6354219317436218
Epoch : 7, Loss : 0.6596015691757202
Epoch : 8, Loss : 0.6621692776679993
Epoch : 9, Loss : 0.6211839914321899
Epoch : 10, Loss : 0.5053560137748718
Epoch : 11, Loss : 0.48787936568260193
Epoch : 12, Loss : 0.4252551198005676
Epoch : 13, Loss : 0.37828874588012695
Epoch : 14, Loss : 0.30432504415512085
Epoch : 15, Loss : 0.27131882309913635
Epoch : 16, Loss : 0.26614078879356384
Epoch : 17, Loss : 0.12729336321353912
Epoch : 18, Loss : 0.08661060035228729
Epoch : 19, Loss : 0.10278407484292984
Epoch : 20, Loss : 0.07161424309015274
Epoch : 21, Loss : 0.04994308203458786
Epoch : 22, Loss : 0.04830531030893326
Epoch : 23, Loss : 0.023128675296902657
Epoch : 24, Loss : 0.015191095881164074
Epoch : 25, Loss : 0.004801814444363117
Epoch : 26, Loss : 0.010499348863959312
Epoch 

In [107]:
test_image = []
from PIL import Image
for image_path in os.listdir(test_dir):
    if image_path == ".DS_Store":
        continue
#         print(image_path)
    curr_image = Image.open(os.path.join(test_dir, image_path))
    curr_image = transform(curr_image)
#         curr_image = cv2.imread(os.path.join(path, image_path))
    test_image.append(curr_image)

In [108]:
model.eval()

kaggle_plate_model(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=131072, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [109]:
def get_predicted_class(output_tensor):
    probabilities = F.softmax(output_tensor, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()
    
    return "dirty" if predicted_class == 0 else "cleaned"

In [112]:
preds = []
with torch.no_grad():
    for image in test_image:
        preds.append(get_predicted_class(model(image)))


In [114]:
import pandas as pd
submission_df = pd.read_csv('/kaggle/input/platesv2/sample_submission.csv')
submission_df['label'] = preds
submission_df

,id,label
0,0,dirty
1,1,dirty
2,2,cleaned
3,3,cleaned
4,4,dirty
...,...,...
739,739,dirty
740,740,cleaned
741,741,cleaned
742,742,dirty


In [115]:
submission_df.to_csv('submission.csv', index=False)